In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import seaborn as sns
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

import findspark
# Start Spark Session
findspark.init('C:\Spark\spark-3.2.2-bin-hadoop2.7')
import pyspark
from pyspark import SparkFiles
from pyspark.sql.functions import to_date, to_timestamp
from getpass import getpass


# Build Spark Session
### IMPORTANT: MAKE SURE THAT spark.config has memory set to 32GB or larger!
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Qwest-Analytics-Dashboard-and-ML-Model").config("spark.jars", "C:\Spark\spark-3.2.2-bin-hadoop2.7\jars\postgresql-42.5.0.jar").getOrCreate()

In [ ]:
# Load the minute aggregation data

minute_aggregation_data_df = spark.read.format("jdbc").options(
         url='jdbc:postgresql://qwest-final-project.ccngkdwtiuvz.us-east-2.rds.amazonaws.com:5432/Qwest-Database',
         dbtable='cleaned_merged_minute_aggregation',
         user='postgres',
         password=password,
         driver='org.postgresql.Driver').load()

minute_aggregation_data_df.show()
minute_aggregation_data_df.printSchema()

In [ ]:
minute_aggregation_data_df = minute_aggregation_data_df.filter(minute_aggregation_data_df('country') === 'United Kingdom')
minute_aggregation_data_df = minute_aggregation_data_df.drop('Unnamed: 0','operator', 'channel', 'cleaned_content_id')
